In [101]:
import numpy as np
import pandas as pd
import json

In [102]:
  #Grab cosine_similarity csv and turn it to dictionary with average similarity score for each product
  def cosine_sim_convertor(name):
    cos_sim = pd.read_csv(name)
    cos_sim = cos_sim.groupby('product_name').agg({
      'similarity_score': np.mean
    })
    cos_sim = cos_sim.to_dict('dict')
    cos_sim_final = cos_sim['similarity_score']
    return cos_sim_final

In [103]:
#grabs all senti scores from json and turn it into dictionary with average senti score for each product
def senti_score_convertor(name):
  file = open(name)
  reviews = json.load(file)
  file.close()
  beer_names = [name for name in reviews]
  d = {}
  for name in beer_names:
    senti = []
    for item in reviews[name]:
      senti.append(item['sentiment']['compound'])
    d[name] = np.array(senti).mean() 
  return d

In [108]:
#consolidates senti scores and review scores into one dataframe
def sim_senti_consolidate(cos_sim,senti):
  d = {}
  for item in beer_names:
    d[item] = {'sentiment':senti[item], 'similarity': cos_sim[item]}
  final_table = pd.DataFrame(d).T
  final_table.index.name = 'name'
  return final_table

In [111]:
#Assume that the data will be in the form of a table that has the following columns: product name, product review, similarity score, sentiment score
#Outputs top 3 products with highest sum of average of sentiment and and average of similarity score 
def consolidate_table(table):
  table['sentsim'] = table['similarity']+table['sentiment']
  return table.sort_values('sentsim',ascending=False).head(3)


In [112]:
review_scores = cosine_sim_convertor('cosine_similarity.csv')
senti_scores = senti_score_convertor('reviews.json')
final_table = sim_senti_consolidate(review_scores,senti_scores)
df_average = consolidate_table(final_table)
print(df_average)

                                           sentiment  similarity   sentsim
name                                                                      
Expedition Stout - Bourbon Barrel-Aged      0.900776    0.011077  0.911853
Cable Car Kriek                             0.911168    0.000546  0.911714
Genealogy Of Morals - Bourbon Barrel-Aged   0.904744    0.001557  0.906301
